# Chrome AI Built-in Prompt API Demo

This notebook demonstrates using Chrome's built-in AI (Gemini Nano) with the Prompt API.

**Requirements:**
- Chrome browser version 128+ 
- Enable experimental flags:
  - `chrome://flags/#prompt-api-for-gemini-nano`
  - `chrome://flags/#optimization-guide-on-device-model`
- First use requires downloading the Gemini Nano model

In [ ]:
import micropip
await micropip.install("anywidget")

In [ ]:
import anywidget
import traitlets

class ChromeAIWidget(anywidget.AnyWidget):
    _esm = """
    async function render({ model, el }) {
      // Create container
      const container = document.createElement("div");
      container.className = "chrome-ai-container";
      
      // Create status message
      const status = document.createElement("div");
      status.className = "status-message";
      status.textContent = "Initializing...";
      
      // Create input box
      const input = document.createElement("textarea");
      input.className = "prompt-input";
      input.placeholder = "Enter your prompt here...";
      input.rows = 3;
      
      // Create submit button
      const button = document.createElement("button");
      button.className = "submit-button";
      button.textContent = "Submit";
      
      // Create response box
      const response = document.createElement("div");
      response.className = "response-box";
      response.textContent = "Response will appear here...";
      
      // Append elements
      container.appendChild(status);
      container.appendChild(input);
      container.appendChild(button);
      container.appendChild(response);
      el.appendChild(container);
      
      // Initialize Chrome AI session
      let session = null;
      
      try {
        // Check if AI is available
        if (!window.ai || !window.ai.languageModel) {
          status.textContent = "Chrome AI not available. Please enable chrome://flags/#prompt-api-for-gemini-nano";
          status.style.color = "#dc2626";
          button.disabled = true;
          return;
        }
        
        // Create session
        status.textContent = "Creating AI session...";
        session = await window.ai.languageModel.create();
        status.textContent = "Ready! Enter a prompt and press Submit or Enter.";
        status.style.color = "#16a34a";
        
      } catch (error) {
        status.textContent = `Error: ${error.message}`;
        status.style.color = "#dc2626";
        button.disabled = true;
        return;
      }
      
      // Handle prompt submission
      async function submitPrompt() {
        const prompt = input.value.trim();
        if (!prompt || !session) return;
        
        try {
          button.disabled = true;
          input.disabled = true;
          status.textContent = "Generating response...";
          status.style.color = "#2563eb";
          response.textContent = "";
          
          // Update Python state
          model.set("prompt", prompt);
          model.save_changes();
          
          // Get AI response
          const result = await session.prompt(prompt);
          
          // Display response
          response.textContent = result;
          
          // Update Python state
          model.set("response", result);
          model.save_changes();
          
          status.textContent = "Response received! Enter another prompt.";
          status.style.color = "#16a34a";
          
        } catch (error) {
          response.textContent = `Error: ${error.message}`;
          status.textContent = "Error occurred. Please try again.";
          status.style.color = "#dc2626";
        } finally {
          button.disabled = false;
          input.disabled = false;
        }
      }
      
      // Button click handler
      button.addEventListener("click", submitPrompt);
      
      // Enter key handler
      input.addEventListener("keydown", (e) => {
        if (e.key === "Enter" && !e.shiftKey) {
          e.preventDefault();
          submitPrompt();
        }
      });
    }
    export default { render };
    """
    
    _css = """
    .chrome-ai-container {
      font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
      max-width: 800px;
      margin: 0 auto;
      padding: 20px;
    }
    
    .status-message {
      padding: 12px;
      margin-bottom: 16px;
      border-radius: 6px;
      background-color: #f3f4f6;
      font-size: 14px;
      font-weight: 500;
    }
    
    .prompt-input {
      width: 100%;
      padding: 12px;
      margin-bottom: 12px;
      border: 2px solid #d1d5db;
      border-radius: 6px;
      font-size: 14px;
      font-family: inherit;
      resize: vertical;
      box-sizing: border-box;
    }
    
    .prompt-input:focus {
      outline: none;
      border-color: #3b82f6;
    }
    
    .submit-button {
      padding: 10px 20px;
      margin-bottom: 16px;
      background-color: #3b82f6;
      color: white;
      border: none;
      border-radius: 6px;
      font-size: 14px;
      font-weight: 500;
      cursor: pointer;
      transition: background-color 0.2s;
    }
    
    .submit-button:hover:not(:disabled) {
      background-color: #2563eb;
    }
    
    .submit-button:disabled {
      background-color: #9ca3af;
      cursor: not-allowed;
    }
    
    .response-box {
      padding: 16px;
      min-height: 100px;
      border: 2px solid #d1d5db;
      border-radius: 6px;
      background-color: #ffffff;
      white-space: pre-wrap;
      word-wrap: break-word;
      font-size: 14px;
      line-height: 1.6;
    }
    """
    
    prompt = traitlets.Unicode("").tag(sync=True)
    response = traitlets.Unicode("").tag(sync=True)

## Create and Display the Widget

Create an instance of the Chrome AI widget and display it:

In [ ]:
ai_widget = ChromeAIWidget()
ai_widget

## Access Prompt and Response from Python

You can access the prompt and response values from Python:

In [ ]:
print("Last prompt:", ai_widget.prompt)
print("Last response:", ai_widget.response)